In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark

#### Create New Spark Session

In [7]:
sc.stop()

In [8]:
from pyspark import SparkConf, SparkContext
#setMaster()- Set Spark Content Manager
config= SparkConf().setMaster("local[2]").setAppName("RDDSession")
sc= SparkContext(conf=config)

In [9]:
sc

<SparkContext master=local[2] appName=RDDSession>

In [11]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("SQlSession").getOrCreate()

#### 2. Create RDD
    * Create RDD using sc.parallelize() using colections like numpy array, lists, tuples etc

In [12]:
rdd1 = sc.parallelize([10,20,30,40,50,60,70,80,90])

In [14]:
# sc.parallelize() created spark rdd object
print(rdd1)

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195


In [15]:
type(rdd1)

pyspark.rdd.RDD

In [18]:
print(rdd1.collect())  

[10, 20, 30, 40, 50, 60, 70, 80, 90]

In [19]:
print(rdd1.take(5))     # take(num of values as per the input count)

[10, 20, 30, 40, 50]


In [20]:
print("Total count of RDD: ",rdd1.count())            # Return total count of values present in RDD

9

In [24]:
rdd1.getNumPartitions()                        # NUmber of RDD partitions = number of workers

2

## Narrow Transformation

In [25]:
#Map transformation - Applies to all individual values
rdd2= rdd1.map(lambda val : val*3)

In [27]:
# Call action() method - collect() or take()
rdd2.collect()

[30, 60, 90, 120, 150, 180, 210, 240, 270]

In [29]:
rdd3=rdd1.map(lambda val : str(val)+"Number")
rdd3.collect()

['10Number',
 '20Number',
 '30Number',
 '40Number',
 '50Number',
 '60Number',
 '70Number',
 '80Number',
 '90Number']

In [38]:
# Filter method - Narrow Transformation which applies condition to each element of RDD
# ALl transformation method like map() or filter() expect a function as an input
rdd4= rdd1.filter(lambda x: x<=70)
rdd4.collect()

[10, 20, 30, 40, 50, 60, 70]

In [42]:
# Create an RDD using range()
rdd5 = sc.parallelize(range(1,30))
rdd5.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [47]:
rdd6 = rdd5.filter(lambda x: x%2==0).map(lambda x : str(x) + ':Even')
print(rdd6.collect())

['2:Even', '4:Even', '6:Even', '8:Even', '10:Even', '12:Even', '14:Even', '16:Even', '18:Even', '20:Even', '22:Even', '24:Even', '26:Even', '28:Even']


In [52]:
city_rdd= sc.parallelize(["Delhi,Kolkata,Kochi,Vizag,Varkala,Chennai,Bangalore,Pune",
                         "Dubai,New York,Berlin,Noida,Bangalore,Vizag",
                         "Venice,Dehradun,Munnar,Mumbai,Kochi,Kottayam"])

In [55]:
city_names=city_rdd.map(lambda x: x.split(','))

In [69]:
city_names_2=city_names.flatMap(lambda val:val)
city_names_2.collect()

['Delhi',
 'Kolkata',
 'Kochi',
 'Vizag',
 'Varkala',
 'Chennai',
 'Bangalore',
 'Pune',
 'Dubai',
 'New York',
 'Berlin',
 'Noida',
 'Bangalore',
 'Vizag',
 'Venice',
 'Dehradun',
 'Munnar',
 'Mumbai',
 'Kochi',
 'Kottayam']

In [73]:
#Union - Union of 2 RDD
rdd7 = rdd1.filter(lambda val : val>20 and val<30)

unionRDD=rdd1.union(rdd7)
unionRDD.collect()

[10, 20, 30, 40, 50, 60, 70, 80, 90]

**#Create rdd using city_names and generate city names starting with "B" or "K"**

In [77]:
city_names_3 = city_names_2.filter(lambda x: x[0].lower()=="b" or x[0].lower()=="k")
city_names_3.collect()

['Kolkata', 'Kochi', 'Bangalore', 'Berlin', 'Bangalore', 'Kochi', 'Kottayam']

In [79]:
#alternate method
city_names_3 = city_names_2.filter(lambda x: x.startswith('K') or x.startswith('B'))
city_names_3.collect().

['Kolkata', 'Kochi', 'Bangalore', 'Berlin', 'Bangalore', 'Kochi', 'Kottayam']

In [117]:
# Sample(without Replacement, fraction, seed)
# Samples a fraction of data from a RDD with replacement
sampled_rdd= city_names_2.sample(False,0.5,seed=42)
sampled_rdd.collect()

['Delhi',
 'Kolkata',
 'Vizag',
 'Varkala',
 'Pune',
 'Vizag',
 'Venice',
 'Dehradun',
 'Munnar',
 'Mumbai',
 'Kochi']

In [120]:
# Sample(with Replacement, fraction, seed)
# Samples a fraction of data from a RDD with replacement
sampled_rdd1= city_names_2.sample(True,0.5,seed=42)
sampled_rdd1.collect()

['Chennai', 'New York', 'Berlin', 'Berlin', 'Munnar', 'Kochi']

In [154]:
# sc.parallelize(collections, number of partitions)
rdd10=sc.parallelize(range(1,50),4)
rdd10.getNumPartitions()

4

In [157]:
def partition_func(iterator):
    return [sum(iterator)]

In [158]:
partitioned_rdd=rdd10.mapPartitions(partition_func)
partitioned_rdd.collect()

[78, 222, 366, 559]

#### Wide Transformation

In [121]:
city_names_3.getNumPartitions()

2

In [122]:
pairs=[('Trivandrum',24),('Chennai',30),('Mumbai',21),
       ('Kochi',12),('Chennai',20),('Trivandrum',15),('Delhi',25),('Delhi',12),('Kottayam',60)]
pairs_rdd=sc.parallelize(pairs)

In [123]:
pairs_rdd.getNumPartitions()

2

In [124]:
#repartition() - To create more number of partition
new_pair_rdd=pairs_rdd.repartition(4)

In [125]:
new_pair_rdd.getNumPartitions()

4

In [129]:
#coalesc() - combines existing RDD Partitions into less number of Partitions
new_pair_rdd1=new_pair_rdd.coalesce(2)
new_pair_rdd1.getNumPartitions()

2

In [135]:
# ReduceByKey() - Aggregate Each Value Of RDD. It does involves shuffling of values to 
#ensure that all values with same key are brought togethor.
pairs_rdd=pairs_rdd.reduceByKey(lambda x,y: x+y)
pairs_rdd.collect()

[('Chennai', 50),
 ('Mumbai', 21),
 ('Kochi', 12),
 ('Kottayam', 60),
 ('Trivandrum', 39),
 ('Delhi', 37)]

In [139]:
orders= [('Fries',2),('Nuggets',3),('PaniPuri',5),('Chole Bhature',2),
         ('Dosa',1),('PaniPuri',5),('Vada','2'),('Chole Bhature',2)]

order_rdd=sc.parallelize(orders)
order_rdd.reduceByKey(lambda x,y : x+y).collect()

[('Nuggets', 3),
 ('Chole Bhature', 4),
 ('Dosa', 1),
 ('Vada', '2'),
 ('Fries', 2),
 ('PaniPuri', 10)]

In [141]:
# Intersection: Method to scan all rdd and find common among all partitions of RDD
rdd8=sc.parallelize(['Delhi','Kolkata','Kochi','Vizag','Varkala','Chennai','Bangalore','Pune'])
rdd9=sc.parallelize(['Bangalore','Pune','Delhi','Vizag','Mumbai','Trivandrum'])
rdd8.intersection(rdd9).collect()

['Pune', 'Delhi', 'Vizag', 'Bangalore']

In [159]:
pairs=[('Trivandrum',24),('Chennai',30),('Mumbai',21),
       ('Kochi',12),('Chennai',20),('Trivandrum',15),('Delhi',25),('Delhi',12),('Kottayam',60)]
pairs_rdd=sc.parallelize(pairs)

In [162]:
# groupByKey()- Group data by key, which involves shuffling all values for each key
grouped_rdd=pairs_rdd.groupByKey().mapValues(tuple)
grouped_rdd.collect()

[('Chennai', (30, 20)),
 ('Mumbai', (21,)),
 ('Kochi', (12,)),
 ('Kottayam', (60,)),
 ('Trivandrum', (24, 15)),
 ('Delhi', (25, 12))]

In [164]:
# len() function returns count of city names.
pairs_rdd.groupByKey().mapValues(len).collect()

[('Chennai', 2),
 ('Mumbai', 1),
 ('Kochi', 1),
 ('Kottayam', 1),
 ('Trivandrum', 2),
 ('Delhi', 2)]

In [2]:
#join() - Joins two RDD based on their keys which requireds shuffling the data to ensure all keys are correctly
#paired
order1= sc.parallelize([('Fries',2),('Nuggets',3),('PaniPuri',5),("Dosa",1),('Vada',2)])
order2= sc.parallelize([('Juice',2),('Idly',3),('PaniPuri',6),("Dosa",1),('Sweets',2),('Biryani',1)])


In [175]:
joined_rdd=order1.join(order2)
joined_rdd.collect()

[('Dosa', (1, 1)), ('PaniPuri', (5, 6))]

In [176]:
order1.leftOuterJoin(order2).collect()

[('Dosa', (1, 1)),
 ('Vada', (2, None)),
 ('PaniPuri', (5, 6)),
 ('Nuggets', (3, None)),
 ('Fries', (2, None))]

In [177]:
order1.rightOuterJoin(order2).collect()

[('Dosa', (1, 1)),
 ('Idly', (None, 3)),
 ('PaniPuri', (5, 6)),
 ('Biryani', (None, 1)),
 ('Juice', (None, 2)),
 ('Sweets', (None, 2))]

In [178]:
order1.fullOuterJoin(order2).collect()

[('Dosa', (1, 1)),
 ('Vada', (2, None)),
 ('Idly', (None, 3)),
 ('PaniPuri', (5, 6)),
 ('Nuggets', (3, None)),
 ('Biryani', (None, 1)),
 ('Fries', (2, None)),
 ('Juice', (None, 2)),
 ('Sweets', (None, 2))]

In [3]:
# Cogroup : Groups data from two rdd by key resulting into new RDD with keys and values grouped from both RDD
results=order1.cogroup(order2).mapValues(tuple).collect()
results

In [7]:
for key, (value1,value2) in results:
    print(f"Key: {key} :{list(value1)},{list(value2)}")

Key: Dosa :[1],[1]
Key: Vada :[2],[]
Key: Idly :[],[3]
Key: PaniPuri :[5],[6]
Key: Nuggets :[3],[]
Key: Biryani :[],[1]
Key: Fries :[2],[]
Key: Juice :[],[2]
Key: Sweets :[],[2]


In [97]:
# distinct = Returns new RDD with distinct elements
distinct_city = city_names_2.distinct()
distinct_city.collect()

NameError: name 'city_names_2' is not defined

**RDD Using FileName()**

In [12]:
# load localFS use "file://path to filename"
emp_RDD= sc.textFile("file:///home//hadoop//Downloads/Employee_Advance.csv")
emp_RDD.take(6)

['1,Binnie,Hovee,bhovee0@hubpages.com,Genderqueer,Support,1419125,Lithuanian,Paris 11,France',
 '2,Devondra,Rosingdall,drosingdall1@yellowbook.com,Non-binary,Support,1365025,Tsonga,Seminole,United States',
 '3,Siward,Struijs,sstruijs2@wikia.com,Male,Business Development,432657,Armenian,London,United States',
 '4,Jaime,Camoys,jcamoys3@wsj.com,Bigender,Accounting,1122884,Armenian,Stockton,United States',
 '5,Nahum,Macieiczyk,nmacieiczyk4@reference.com,Bigender,Human Resources,1026666,Burmese,Toulouse,France',
 '6,Tova,Manclark,tmanclark5@army.mil,Female,Business Development,659039,Hiri Motu,Le Mans,France']

In [13]:
emp_RDD1= emp_RDD.map(lambda row: row.split(','))
emp_RDD1.collect()

[['1',
  'Binnie',
  'Hovee',
  'bhovee0@hubpages.com',
  'Genderqueer',
  'Support',
  '1419125',
  'Lithuanian',
  'Paris 11',
  'France'],
 ['2',
  'Devondra',
  'Rosingdall',
  'drosingdall1@yellowbook.com',
  'Non-binary',
  'Support',
  '1365025',
  'Tsonga',
  'Seminole',
  'United States'],
 ['3',
  'Siward',
  'Struijs',
  'sstruijs2@wikia.com',
  'Male',
  'Business Development',
  '432657',
  'Armenian',
  'London',
  'United States'],
 ['4',
  'Jaime',
  'Camoys',
  'jcamoys3@wsj.com',
  'Bigender',
  'Accounting',
  '1122884',
  'Armenian',
  'Stockton',
  'United States'],
 ['5',
  'Nahum',
  'Macieiczyk',
  'nmacieiczyk4@reference.com',
  'Bigender',
  'Human Resources',
  '1026666',
  'Burmese',
  'Toulouse',
  'France'],
 ['6',
  'Tova',
  'Manclark',
  'tmanclark5@army.mil',
  'Female',
  'Business Development',
  '659039',
  'Hiri Motu',
  'Le Mans',
  'France'],
 ['7',
  'Terrill',
  'Feander',
  'tfeander6@unc.edu',
  'Genderqueer',
  'Legal',
  '226156',
  'Greek'

#### 1. Show all employee working is Department = "Business Development"

In [14]:
emp_BD=emp_RDD1.filter(lambda row: row[5]=="Business Development")
emp_BD.collect()

[['3',
  'Siward',
  'Struijs',
  'sstruijs2@wikia.com',
  'Male',
  'Business Development',
  '432657',
  'Armenian',
  'London',
  'United States'],
 ['6',
  'Tova',
  'Manclark',
  'tmanclark5@army.mil',
  'Female',
  'Business Development',
  '659039',
  'Hiri Motu',
  'Le Mans',
  'France'],
 ['9',
  'Warde',
  'Stenett',
  'wstenett8@altervista.org',
  'Genderqueer',
  'Business Development',
  '807769',
  'Moldovan',
  'Bastia',
  'France'],
 ['17',
  'Bern',
  'Lafond',
  'blafondg@amazon.de',
  'Non-binary',
  'Business Development',
  '933737',
  'Greek',
  'San Antonio',
  'United States'],
 ['19',
  'Hillery',
  'Costall',
  'hcostalli@rakuten.co.jp',
  'Genderfluid',
  'Business Development',
  '736463',
  'Norwegian',
  'Clermont-Ferrand',
  'France'],
 ['22',
  'Trish',
  'Walework',
  'twaleworkl@devhub.com',
  'Agender',
  'Business Development',
  '1208531',
  'Bislama',
  'Lees Summit',
  'United States'],
 ['56',
  'Deidre',
  'Monckton',
  'dmonckton1j@sakura.ne.jp

In [15]:
# Return first 5 rows for list of Employee working in Business Development.
emp_BD.take(5)

[['3',
  'Siward',
  'Struijs',
  'sstruijs2@wikia.com',
  'Male',
  'Business Development',
  '432657',
  'Armenian',
  'London',
  'United States'],
 ['6',
  'Tova',
  'Manclark',
  'tmanclark5@army.mil',
  'Female',
  'Business Development',
  '659039',
  'Hiri Motu',
  'Le Mans',
  'France'],
 ['9',
  'Warde',
  'Stenett',
  'wstenett8@altervista.org',
  'Genderqueer',
  'Business Development',
  '807769',
  'Moldovan',
  'Bastia',
  'France'],
 ['17',
  'Bern',
  'Lafond',
  'blafondg@amazon.de',
  'Non-binary',
  'Business Development',
  '933737',
  'Greek',
  'San Antonio',
  'United States'],
 ['19',
  'Hillery',
  'Costall',
  'hcostalli@rakuten.co.jp',
  'Genderfluid',
  'Business Development',
  '736463',
  'Norwegian',
  'Clermont-Ferrand',
  'France']]

2.Find Average Income Of Each Department

In [62]:
dep_income=emp_RDD1.map(lambda x: (x[5],int(x[6])))
income_list=dep_income.groupByKey().mapValues(list)
avg_income=income_list.mapValues(lambda x: sum(x)/len(x))
avg_income.collect()


[('Accounting', 814474.3333333334),
 ('Human Resources', 772824.3829787234),
 ('Legal', 754482.0),
 ('Services', 765440.4444444445),
 ('Sales', 745501.5921052631),
 ('Product Management', 696304.2985074627),
 ('Support', 825695.0947368421),
 ('Business Development', 783081.2710280374),
 ('Research and Development', 789849.1486486486),
 ('Training', 757752.380952381),
 ('Marketing', 744170.7831325302),
 ('Engineering', 757094.5466666666)]

In [96]:
#Alternate
emp_Income=emp_RDD1.map(lambda row: (row[5],int(row[6])))
emp_Map=emp_RDD1.map(lambda row: (int(row[6])))
emp_income1=emp_Income.reduceByKey(lambda x,y: x+y)
emp_income2=emp_Map.reduceByKey(lambda x,y: x+y)
emp_income1.join(emp_income2.mapValues())
??

NameError: name 'emp_Map' is not defined

3. Count the total number of employees working in organisation

In [95]:
total_count=emp_RDD1.map(lambda x:("count",1)).reduceByKey(lambda x,y:x+y)
total_count.collect()

[('count', 1000)]

4. Count the Number of Employee Working in Each Department

In [38]:
dept_inc = emp_RDD1.map(lambda val: (val[5], 1)).reduceByKey(lambda x,y: x+y)
dept_inc.collect()

[('Accounting', 93),
 ('Human Resources', 94),
 ('Legal', 80),
 ('Services', 72),
 ('Sales', 76),
 ('Product Management', 67),
 ('Support', 95),
 ('Business Development', 107),
 ('Research and Development', 74),
 ('Training', 84),
 ('Marketing', 83),
 ('Engineering', 75)]

In [42]:
#Alternative method
emp_RDD1.groupBy(lambda row: row[5]).mapValues(len).collect()

[('Accounting', 93),
 ('Human Resources', 94),
 ('Legal', 80),
 ('Services', 72),
 ('Sales', 76),
 ('Product Management', 67),
 ('Support', 95),
 ('Business Development', 107),
 ('Research and Development', 74),
 ('Training', 84),
 ('Marketing', 83),
 ('Engineering', 75)]

5. List all unique Job in Dataset

In [92]:
e=emp_RDD1.map(lambda x: x[5]).distinct()
e.collect()

['Accounting',
 'Human Resources',
 'Legal',
 'Services',
 'Sales',
 'Product Management',
 'Support',
 'Business Development',
 'Research and Development',
 'Training',
 'Marketing',
 'Engineering']

6. Count all employees whose first letter starts with a specific letter A

In [90]:
e=emp_RDD1.filter(lambda x:x[1][0]=='A').map(lambda x: ("Starting_A",1)).reduceByKey(lambda x,y: x+y)
e.collect()

[('Starting_A', 81)]

In [100]:
#alternate
emp_RDD1.filter(lambda row: row[1].startswith('A')).count()

81

#### 7) Word Count Example using Spark RDD

In [122]:
!hdfs dfs -ls /

Found 4 items
drwxr-xr-x   - hadoop supergroup          0 2022-11-21 15:25 /hbase
drwxrwxrwx   - hadoop supergroup          0 2022-11-21 15:12 /tmp
drwxr-xr-x   - hadoop supergroup          0 2022-11-21 15:11 /user
drwxr-xr-x   - hadoop supergroup          0 2024-08-31 12:11 /wordcount


In [119]:
!hdfs dfs -mkdir /wordcount

mkdir: `/wordcount': File exists


In [123]:
!hdfs dfs -copyFromLocal /home/hadoop/Downloads/Harry_Potter_and_the_Deathly_Hallows.txt /wordcount
#instead of copyFromLocal we can also use put

In [126]:
#rdd=sc.textFile('file://path-of-file')    # file://path-of-file used when file in local system

rdd=sc.textFile('/wordcount/Harry_Potter_and_the_Deathly_Hallows.txt')
rdd.collect()

['Chapter 1',
 'The Dark Lord Ascending',
 '',
 'The two men appeared out of nowhere, a few yards apart in the narrow, moonlit lane. For a second they stood quite still, wands directed at each other’s chests; then, recognizing each other, they stowed their wands beneath their cloaks and started walking briskly in the same direction.',
 '“News?” asked the taller of the two.',
 '“The best,” replied Severus Snape.',
 'The lane was bordered on the left by wild, low-growing brambles, on the right by a high, neatly manicured hedge. The men’s long cloaks flapped around their ankles as they marched.',
 '“Thought I might be late,” said Yaxley, his blunt features sliding in and out of sight as the branches of overhanging trees broke the moonlight. “It was a little trickier than I expected. But I hope he will be satisfied. You sound confident that your reception will be good?”',
 'Snape nodded, but did not elaborate. They turned right, into a wide driveway that led off the lane. The high hedge cu

In [127]:
rdd_word1=rdd.flatMap(lambda line: line.split(' '))
rdd_word2=rdd_word1.map(lambda word : (word.lower(),1))
rdd_word3=rdd_word2.reduceByKey(lambda x,y: x+y)
rdd_word4=rdd_word3.filter(lambda val: val[1]>10).sortBy(lambda val: val[1],ascending=False)
rdd_word4.collect()

[('the', 10280),
 ('and', 5283),
 ('to', 4843),
 ('of', 4121),
 ('he', 3845),
 ('a', 3523),
 ('was', 2681),
 ('his', 2602),
 ('in', 2159),
 ('had', 1990),
 ('harry', 1867),
 ('it', 1859),
 ('that', 1813),
 ('said', 1676),
 ('you', 1485),
 ('as', 1409),
 ('at', 1401),
 ('i', 1275),
 ('with', 1127),
 ('they', 1091),
 ('not', 1083),
 ('on', 984),
 ('for', 980),
 ('but', 975),
 ('she', 918),
 ('her', 864),
 ('—', 841),
 ('from', 836),
 ('have', 784),
 ('…', 756),
 ('were', 738),
 ('be', 737),
 ('him', 666),
 ('into', 655),
 ('hermione', 653),
 ('out', 650),
 ('could', 627),
 ('all', 583),
 ('ron', 578),
 ('what', 571),
 ('been', 567),
 ('“i', 535),
 ('up', 510),
 ('we', 498),
 ('there', 493),
 ('who', 490),
 ('is', 484),
 ('—”', 472),
 ('their', 465),
 ('did', 451),
 ('would', 443),
 ('them', 442),
 ('like', 424),
 ('if', 424),
 ('looked', 419),
 ('by', 419),
 ('harry,', 414),
 ('back', 409),
 ('so', 403),
 ('over', 383),
 ('this', 382),
 ('an', 381),
 ('then', 365),
 ('one', 364),
 ('wand

#### 8) Count the number of employees working in each city and sorty city by Employees count in descending order

In [133]:
emp_RDD1.map(lambda x: (x[8],1)).groupByKey().mapValues(len).sortBy(lambda val: val[1],ascending=False).collect()

[('Washington', 13),
 ('Berlin', 13),
 ('Lyon', 13),
 ('Cincinnati', 9),
 ('Dallas', 9),
 ('Cergy-Pontoise', 9),
 ('Sacramento', 8),
 ('Strasbourg', 8),
 ('Lille', 7),
 ('Dijon', 7),
 ('Los Angeles', 7),
 ('Pittsburgh', 7),
 ('Bordeaux', 7),
 ('San Jose', 7),
 ('Denver', 7),
 ('El Paso', 7),
 ('Montpellier', 7),
 ('Chicago', 6),
 ('Tulsa', 6),
 ('Quimper', 6),
 ('Marseille', 6),
 ('New York City', 6),
 ('Pau', 6),
 ('Poitiers', 6),
 ('Montgomery', 6),
 ('Le Mans', 5),
 ('Avignon', 5),
 ('Seattle', 5),
 ('Marne-la-Vallée', 5),
 ('Limoges', 5),
 ('Oakland', 5),
 ('Fort Worth', 5),
 ('Houston', 5),
 ('San Diego', 5),
 ('Philadelphia', 5),
 ('Paris La Défense', 5),
 ('Rungis', 5),
 ('Perpignan', 5),
 ('Rouen', 5),
 ('Caen', 5),
 ('Minneapolis', 5),
 ('Nantes', 5),
 ('Orléans', 5),
 ('Nürnberg', 4),
 ('Mobile', 4),
 ('Aix-en-Provence', 4),
 ('Angoulême', 4),
 ('Charlotte', 4),
 ('Düsseldorf', 4),
 ('Topeka', 4),
 ('Kansas City', 4),
 ('Roissy Charles-de-Gaulle', 4),
 ('Saint Petersburg', 4)

#### Saving Data Using RDD

In [134]:
rdd_word4.getNumPartitions()

2

In [136]:
#Saving RDD data to local file system
rdd_word4.saveAsTextFile('file:///home/hadoop/Downloads/output/')

In [140]:
#Saving RDD data to local filesystem
rdd_word4.saveAsTextFile('/wordcount/output/')

####  RDD -Satistical Function


In [141]:
rdd_City=emp_RDD1.map(lambda row : (row[5],row[6]))

In [142]:
rdd_City.collect()

[('Support', '1419125'),
 ('Support', '1365025'),
 ('Business Development', '432657'),
 ('Accounting', '1122884'),
 ('Human Resources', '1026666'),
 ('Business Development', '659039'),
 ('Legal', '226156'),
 ('Research and Development', '545708'),
 ('Business Development', '807769'),
 ('Human Resources', '235352'),
 ('Training', '83107'),
 ('Support', '1008502'),
 ('Legal', '685235'),
 ('Training', '374701'),
 ('Human Resources', '1261986'),
 ('Services', '393009'),
 ('Business Development', '933737'),
 ('Services', '758842'),
 ('Business Development', '736463'),
 ('Human Resources', '978985'),
 ('Accounting', '866039'),
 ('Business Development', '1208531'),
 ('Sales', '569996'),
 ('Marketing', '895390'),
 ('Marketing', '137077'),
 ('Human Resources', '446416'),
 ('Support', '44465'),
 ('Legal', '273729'),
 ('Training', '1316779'),
 ('Marketing', '392291'),
 ('Research and Development', '1148022'),
 ('Legal', '1197458'),
 ('Engineering', '438302'),
 ('Training', '1409971'),
 ('Training

In [149]:
rdd_City.min(lambda x : int(x[1]))

('Training', '12727')

In [148]:
rdd_City.max(lambda x : int(x[1]))

('Support', '1496924')

In [176]:
rdd_11=sc.parallelize(range(1,100)).sample(False,0.3,seed=2)
rdd_11.min()

16

In [195]:
rdd_City.map(lambda x: int(x[1])).stdev()

424405.95187845733

In [ ]:
rdd_City.map(lambda x: int(x[1])).stdev()

In [205]:
rdd_City.map(lambda x: int(x[1])).variance()

180120411989.85944

In [203]:
rdd_City.map(lambda x: int(x[1])).mean()

770051.4190000002

In [204]:
rdd_City.map(lambda x: int(x[1])).sum()

770051419

In [207]:
rdd_City.map(lambda x: int(x[1])).stats()

(count: 1000, mean: 770051.4190000002, stdev: 424405.95187845733, max: 1496924, min: 12727)

In [208]:
emp_RDD.persist("MEMORY ONLY").

SyntaxError: invalid syntax (<ipython-input-208-1311b6bd7845>, line 1)

In [210]:
from pyspark .storagelevel import StorageLevel

In [213]:
# Persist the RDD in memory with default storage level( MEMORY_ONLY)
emp_RDD=emp_RDD.persist(StorageLevel.MEMORY_ONLY)
emp_RDD1=emp_RDD.map(lambda row: row.split(","))
emp_Map=emp_RDD1.map(lambda row: (row[5],1))
emp_Map.reduceByKey(lambda x,y: x+y).collect()

[('Accounting', 93),
 ('Human Resources', 94),
 ('Legal', 80),
 ('Services', 72),
 ('Sales', 76),
 ('Product Management', 67),
 ('Support', 95),
 ('Business Development', 107),
 ('Research and Development', 74),
 ('Training', 84),
 ('Marketing', 83),
 ('Engineering', 75)]

In [217]:
emp_RDD.unpersist()

file:///home//hadoop//Downloads/Employee_Advance.csv MapPartitionsRDD[17] at textFile at NativeMethodAccessorImpl.java:0

In [218]:
emp_RDD= emp_RDD.persist(StorageLevel.MEMORY_AND_DISK_SER)

In [219]:
emp_RDD.unpersist()

file:///home//hadoop//Downloads/Employee_Advance.csv MapPartitionsRDD[17] at textFile at NativeMethodAccessorImpl.java:0